In [3]:
import numpy as np 
import pandas as pd 

import os
import matplotlib.pyplot as plt
import seaborn as sns


In [5]:
df=pd.read_csv(r"C:\Users\DELL\Desktop\CropAI\data\class\Crop_details.csv")

In [6]:
df.head(5)

,Unnamed: 0,path,crop,croplabel
0,0,/kaggle/input/kag2/sugarcane/sugarcane037ahs.jpeg,sugarcane,3
1,1,/kaggle/input/kag2/sugarcane/sugarcane034a.jpeg,sugarcane,3
2,2,/kaggle/input/kag2/sugarcane/sugarcane032arot....,sugarcane,3
3,3,/kaggle/input/kag2/sugarcane/sugarcane036ahs.jpeg,sugarcane,3
4,4,/kaggle/input/kag2/sugarcane/sugarcane023ahs.jpeg,sugarcane,3


In [7]:
df = df.drop(columns=["Unnamed: 0", "crop"])


In [8]:
df.columns

Index(['path', 'croplabel'], dtype='object')

In [10]:
import pandas as pd
import os

# 1. Use absolute paths to avoid "directory not found" errors
base_folder = r"C:\Users\DELL\Desktop\CropAI\data\class"
input_csv = os.path.join(base_folder, "Crop_details.csv")
output_csv = os.path.join(base_folder, "Crop_details_local.csv")

# Load the dataset
df = pd.read_csv(input_csv)

# 2. Fix the path replacement
# We use the absolute base folder path so the images can be found from anywhere
local_base = base_folder + "/" # Adds a slash to the end of your local path

# Replace '/kaggle/input' with your local absolute path
# We use .replace('\\', '/') to keep slashes consistent
df['path'] = df['path'].str.replace('/kaggle/input', local_base, regex=False)
df['path'] = df['path'].str.replace('//', '/') # Clean up any double slashes

# 3. Label Encoding (Mapping crop names to your 0-4 scale)
label_map = {
    'jute': 0,
    'maize': 1,
    'rice': 2,
    'sugarcane': 3,
    'wheat': 4
}
# If your CSV has a 'label' column with text, this creates the numeric 'label' column
if 'label' in df.columns and df['label'].dtype == 'object':
    df['label'] = df['label'].map(label_map)

# 4. Verify paths
sample_path = df.iloc[0]['path']
if os.path.exists(sample_path):
    print(f"✅ Success! Path found: {sample_path}")
else:
    print(f"❌ Warning: Path still not found: {sample_path}")
    print("Double check if 'kag2' is directly inside 'class' or another folder.")

# 5. Save the updated CSV (This will now work because the path is absolute)
df.to_csv(output_csv, index=False)
print(f"📂 Saved updated CSV to: {output_csv}")

✅ Success! Path found: C:\Users\DELL\Desktop\CropAI\data\class/kag2/sugarcane/sugarcane037ahs.jpeg
📂 Saved updated CSV to: C:\Users\DELL\Desktop\CropAI\data\class\Crop_details_local.csv


In [22]:
import pandas as pd

In [24]:
DATA=pd.read_csv(r"C:\Users\DELL\Desktop\CropAI\data\class\Crop_details_local.csv")

In [25]:
DATA.head()

,Unnamed: 0,path,crop,croplabel
0,0,C:\Users\DELL\Desktop\CropAI\data\class/kag2/s...,sugarcane,3
1,1,C:\Users\DELL\Desktop\CropAI\data\class/kag2/s...,sugarcane,3
2,2,C:\Users\DELL\Desktop\CropAI\data\class/kag2/s...,sugarcane,3
3,3,C:\Users\DELL\Desktop\CropAI\data\class/kag2/s...,sugarcane,3
4,4,C:\Users\DELL\Desktop\CropAI\data\class/kag2/s...,sugarcane,3


In [27]:
DATA.drop(columns=['Unnamed: 0','crop'], inplace=True)

In [28]:
DATA.head()

,path,croplabel
0,C:\Users\DELL\Desktop\CropAI\data\class/kag2/s...,3
1,C:\Users\DELL\Desktop\CropAI\data\class/kag2/s...,3
2,C:\Users\DELL\Desktop\CropAI\data\class/kag2/s...,3
3,C:\Users\DELL\Desktop\CropAI\data\class/kag2/s...,3
4,C:\Users\DELL\Desktop\CropAI\data\class/kag2/s...,3


In [29]:
!pip install tensorflow pandas numpy matplotlib scikit-learn opencv-python


In [30]:
import pandas as pd
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam


In [31]:
X = DATA["path"].values
y = DATA["croplabel"].values

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [32]:
IMG_SIZE = 224

def load_images(path):
    images = []
    for paths in path:
        img = cv2.imread(paths)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        img = preprocess_input(img)
        images.append(img)
    return np.array(images)


In [33]:
X_train_img = load_images(X_train)
X_val_img = load_images(X_val)

y_train_cat = to_categorical(y_train, num_classes=5)
y_val_cat = to_categorical(y_val, num_classes=5)


In [34]:
base_model = EfficientNetB0(
    weights="imagenet",
    include_top=False,
    input_shape=(224, 224, 3)
)

base_model.trainable = False  # freeze pretrained layers

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.4)(x)
output = Dense(5, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=output)


16705208/16705208 [==============================] - 4s 0us/step


In [35]:
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 rescaling (Rescaling)       (None, 224, 224, 3)          0         ['input_1[0][0]']             
                                                                                                  
 normalization (Normalizati  (None, 224, 224, 3)          7         ['rescaling[0][0]']           
 on)                                                                                              
                                                                                                  
 rescaling_1 (Rescaling)     (None, 224, 224, 3)          0         ['normalization[0][0]']   

In [36]:
history = model.fit(
    X_train_img,
    y_train_cat,
    validation_data=(X_val_img, y_val_cat),
    epochs=10,
    batch_size=16
)


Epoch 1/10
50/50 [==============================] - 71s 1s/step - loss: 1.3080 - accuracy: 0.4894 - val_loss: 0.9140 - val_accuracy: 0.7650
Epoch 2/10
50/50 [==============================] - 61s 1s/step - loss: 0.7757 - accuracy: 0.7772 - val_loss: 0.6297 - val_accuracy: 0.8600
Epoch 3/10
50/50 [==============================] - 61s 1s/step - loss: 0.5882 - accuracy: 0.8260 - val_loss: 0.4842 - val_accuracy: 0.9000
Epoch 4/10
50/50 [==============================] - 60s 1s/step - loss: 0.4666 - accuracy: 0.8798 - val_loss: 0.3980 - val_accuracy: 0.9100
Epoch 5/10
50/50 [==============================] - 67s 1s/step - loss: 0.3777 - accuracy: 0.9161 - val_loss: 0.3370 - val_accuracy: 0.9400
Epoch 6/10
50/50 [==============================] - 74s 1s/step - loss: 0.3282 - accuracy: 0.9349 - val_loss: 0.2891 - val_accuracy: 0.9600
Epoch 7/10
50/50 [==============================] - 73s 1s/step - loss: 0.2861 - accuracy: 0.9374 - val_loss: 0.2504 - val_accuracy: 0.9700
Epoch 8/10
50/50 [==

In [49]:
base_model.trainable = True

model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

history_finetune = model.fit(
    X_train_img,
    y_train_cat,
    validation_data=(X_val_img, y_val_cat),
    epochs=2,
    batch_size=16
)


Epoch 1/2
50/50 [==============================] - 295s 5s/step - loss: 1.0756 - accuracy: 0.5732 - val_loss: 0.3528 - val_accuracy: 0.9000
Epoch 2/2
50/50 [==============================] - 252s 5s/step - loss: 0.9045 - accuracy: 0.6496 - val_loss: 0.3915 - val_accuracy: 0.8950


In [50]:
model.save("models/crop_classifier.h5")


c:\Users\DELL\Desktop\CropAI\virtualEN\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [53]:
def predict_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    img = preprocess_input(img)
    img = np.expand_dims(img, axis=0)
    
    pred = model.predict(img)
    return np.argmax(pred)

predict_image(r"C:\Users\DELL\Desktop\CropAI\data\class\test_crop_image\sugarcanefield.jpg")


1/1 [==============================] - 0s 113ms/step


3

In [54]:
def predict_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    img = preprocess_input(img)
    img = np.expand_dims(img, axis=0)
    
    pred = model.predict(img)
    return np.argmax(pred)

predict_image(r"C:\Users\DELL\Desktop\CropAI\data\class\test_crop_image\rice-field02.jpg")


1/1 [==============================] - 0s 163ms/step


2

In [60]:

def predict_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    img = preprocess_input(img)
    img = np.expand_dims(img, axis=0)
    
    pred = model.predict(img)
    return np.argmax(pred)

predict_image(r"C:\Users\DELL\Desktop\CropAI\data\class\some_more_images\some_more_images\jute\jute050a.jpg")



1/1 [==============================] - 0s 123ms/step


0